In [258]:
from lib import models, graph, coarsening, utils
import numpy as np
import matplotlib.pyplot as plt
import data_converter as dst
import scipy
%matplotlib inline

In [259]:
curr_data_twitter = dst.get_twitter_data_set()
count_data = len(curr_data_twitter["data"])
print(count_data)
count_labels = len(curr_data_twitter["labels"])
print(count_labels)
assert count_data == count_labels
print(curr_data_twitter["label_values"])
count_value_labels = len(curr_data_twitter["label_values"])

144033
144033
{'-1', '1'}


In [260]:
curr_graphs = curr_data_twitter["data"] 
labels = curr_data_twitter["labels"]
new_graphs = [scipy.sparse.coo_matrix(elem).astype(np.float32) for elem in curr_graphs]
np_new_graphs = np.ndarray((len(new_graphs), 1))
np_new_graphs = np.append(np_new_graphs, new_graphs)

new_y_train = np.ndarray((len(y_train), 1))
new_y_val = np.ndarray((len(y_val), 1))

new_y_train = np.append(new_y_train, y_train)
new_y_val = np.append(new_y_val, y_val)

y_train = new_y_train
y_val = new_y_val
print(new_graphs[0])

  (0, 1)	0.0003034013
  (2, 1)	0.00035130675
  (2, 3)	0.00016500772


In [261]:
n_train = count_data // 2
n_val = count_data // 10

X_train = np_new_graphs[:n_train]
X_val   = np_new_graphs[n_train:n_train+n_val]
X_test  = np_new_graphs[n_train+n_val:]

y_train = labels[:n_train]
y_val   = labels[n_train:n_train+n_val]
y_test  = labels[n_train+n_val:]


In [262]:
L = [graph.laplacian(A, normalized=True).astype(np.float32) for A in new_graphs]

In [263]:
params = dict()
params['dir_name']       = 'demo'
params['num_epochs']     = 40
params['batch_size']     = 1
params['eval_frequency'] = 200

# Building blocks.
params['filter']         = 'chebyshev5'
params['brelu']          = 'b1relu'
params['pool']           = 'apool1'

# Number of classes.
C = count_value_labels

print(len(L))
# Architecture.
params['F']              = [32, 64]  # Number of graph convolutional filters.
params['K']              = [20, 20]  # Polynomial orders.
params['p']              = [1, 2]    # Pooling sizes.
params['M']              = [512, C]  # Output dimensionality of fully connected layers.

# Optimization.
params['regularization'] = 5e-4
params['dropout']        = 1
params['learning_rate']  = 1e-3
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['decay_steps']    = n_train / params['batch_size']

144033


In [264]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
model = models.cgcnn(L, **params)
accuracy, loss, t_step = model.fit(X_train, y_train, X_val, y_val)

(72016,)


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(accuracy, 'b.-')
ax1.set_ylabel('validation accuracy', color='b')
ax2 = ax1.twinx()
ax2.plot(loss, 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

In [ ]:
print('Time per step: {:.2f} ms'.format(t_step*1000))

In [ ]:
res = model.evaluate(X_test, y_test)
print(res[0])